## 1. Necessary Imports 

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn import preprocessing


from skmultiflow.data import HyperplaneGenerator
from scipy.stats import norm
import random
from scipy import stats 


import tensorflow
import tensorflow.keras.backend as K
import tensorflow.keras.layers as layers

import warnings
warnings.filterwarnings('ignore')


## 2.Hyperplane Data Generation

## 3. Reading the saved data 

In [2]:
data=pd.read_csv('hyperplane_nondrifted.csv')

In [3]:
data

,att0,att1,att2,att3,att4,att5,att6,att7,att8,att9,output
0,0.685360,0.953393,0.003948,0.512192,0.812621,0.612526,0.721755,0.291876,0.917774,0.714576,1
1,0.142170,0.373341,0.674134,0.441833,0.434014,0.617767,0.513138,0.650397,0.601039,0.805223,0
2,0.908649,0.319236,0.090459,0.300700,0.113984,0.828681,0.046896,0.626287,0.547586,0.819287,0
3,0.856850,0.351653,0.754648,0.295962,0.883936,0.325512,0.165016,0.392529,0.093460,0.821106,1
4,0.384114,0.944261,0.987625,0.456305,0.826123,0.251374,0.597372,0.902832,0.534558,0.590201,1
...,...,...,...,...,...,...,...,...,...,...,...
39995,0.061714,0.462585,0.943864,0.143777,0.657776,0.325195,0.040887,0.946246,0.313470,0.068780,0
39996,0.073441,0.542813,0.416149,0.585142,0.912097,0.400436,0.582935,0.003902,0.662642,0.179987,0
39997,0.686396,0.536559,0.136439,0.324377,0.606774,0.788541,0.472763,0.443855,0.809508,0.259377,0
39998,0.871673,0.893466,0.046246,0.231087,0.316337,0.445972,0.039865,0.051370,0.502055,0.350849,0


In [4]:
drifted_data=pd.read_csv('hyperplane_drifted.csv')

In [5]:
drifted_data

,att0,att1,att2,att3,att4,att5,att6,att7,att8,att9,output
0,0.855904,0.753047,0.804241,0.623340,0.608495,0.418715,0.162841,0.105965,0.589660,0.829179,1
1,0.784414,0.408043,0.259734,0.855947,0.919578,0.984292,0.186262,0.663626,0.482852,0.981291,1
2,0.658460,0.033933,0.713858,0.378366,0.906390,0.875112,0.190273,0.245564,0.062429,0.841749,1
3,0.136558,0.231648,0.482025,0.693201,0.245718,0.882909,0.635738,0.522973,0.960885,0.520092,0
4,0.152298,0.665623,0.595665,0.101269,0.544626,0.456066,0.155814,0.648949,0.516301,0.073947,0
...,...,...,...,...,...,...,...,...,...,...,...
2555,0.849341,0.442411,0.238075,0.930455,0.851710,0.134304,0.680624,0.941170,0.695621,0.203228,1
2556,0.333330,0.205006,0.313797,0.816728,0.412498,0.847833,0.961956,0.100099,0.973875,0.380310,0
2557,0.695616,0.735272,0.286571,0.622903,0.656901,0.127594,0.368878,0.212568,0.305806,0.933580,1
2558,0.534210,0.876190,0.020611,0.358940,0.932018,0.659357,0.118251,0.273765,0.597341,0.027397,1


## 3. Data Preprocessing 
1. Apply one-hot encoding to object types
2. Apply Minmax normalization to numeric columns 
3. Keep the class column as it is
4. Initial 70% data is used for training ( training and validation for AE)
5. Next 20 % data is used as a validation set to compute thresholds
6. Next 10 % data is used as a test stream initilally without drift 
  

In [6]:
def normalize_encode_split(data,label_col,pos_val,neg_val):        
    # Apply Label Encoding 
    # Apply MinMAx Normalization
    # Split into training  , test (validation set 1) and stream ( non-drifted data stream)   
            
    train=data[0:int(len(data)*0.70)]
    test=data[int((0.70*len(data))):int((0.90*len(data)))]
    stream=data[int((0.90*len(data))):len(data)]
    
    train_positives = train[train[label_col] == pos_val]
    train_negatives = train[train[label_col] == neg_val]
    
    
    y_positive=train_positives[label_col] 
    y_negative=train_negatives[label_col]
    X_positive=train_positives.drop([label_col],axis=1)
    X_negative=train_negatives.drop([label_col],axis=1)
    
    
    return train, test, X_positive,X_negative , stream ,y_positive ,y_negative    
            

In [7]:
train, test, X_positive,X_negative , stream, y_positive ,y_negative =normalize_encode_split(data,'output',1,0)

## 4. AEDDM Paramaters / Hyperparamaters 

## Grid Search Autoencoder Hyperparametrs 

In [8]:
import tensorflow as tf
hyperparameter_space = {
    'hidden_layers':[0,1,2,3,4],
    'optimizer': [tf.keras.optimizers.Adam(), tf.keras.optimizers.SGD()],
    'regularizer':[None,'l1','l2'],
     'learning_rate':[0.001, 0.01, 0.05, 0.1]
    }

In [9]:
import itertools
all_combinations = list(itertools.product(*hyperparameter_space.values()))

In [10]:
len(all_combinations)

120

In [11]:
X_Positive_train=X_positive[0:int(len(X_positive)*0.90)]
X_Positive_test=X_positive[int((0.90*len(X_positive))):len(X_positive)-1]

In [16]:
def grid_search(train,val_set,hyperparameter_space):
    
    
    all_combinations = list(itertools.product(*hyperparameter_space.values()))
    best_hyperparameters = None
    best_loss = np.inf
    
    
    for hyperparameters in all_combinations:
        
        optimizer = hyperparameters[1]
        kernel_regularizer=hyperparameters[2]    
        learning_rate = hyperparameters[3]  # Extract the learning rate
        optimizer.learning_rate = learning_rate
    
        
        if (hyperparameters[0]==0):
        
            # Decide the network architecture 
            print("Size:3 , No Hidden layer")
            input=int(train.shape[1])
            if(input<=5):
                latent=2
            else:    
                latent=int(input/2)             
                latent=latent-1 if (latent%2!=0) else latent
            print(input,latent,input)
        
            # Setup the AE Architecture 
            
             
            input_layer = tensorflow.keras.Input(shape=train.shape[1:]) # Input Layer
            encoded=layers.Dense(latent,activation='relu')(input_layer) # Bottleneck
            decoded = layers.Dense(train.shape[1], activation='sigmoid',kernel_regularizer=kernel_regularizer)(encoded) # Output Layer

            autoencoder = tensorflow.keras.Model(input_layer, decoded)
            # above model maps an input to its reconstruction 
   
            #for hyperparameters in all_combinations:
                # Create and train an autoencoder with the given hyperparameters
            autoencoder.compile(optimizer=optimizer, loss='mse')
            print(hyperparameters)
            #print(hyperparameters[0],hyperparameters[1],hyperparameters[2])
            history=autoencoder.fit(train,train, 
                    epochs=100,
                    batch_size=32,
                    shuffle=True,
                    validation_data=(val_set, val_set)).history
             
            if np.mean(history['val_loss']) < best_loss:
                best_hyperparameters = hyperparameters
                best_loss = np.mean(history['val_loss'])
                best_model=autoencoder
                best_history=history
        
        
        
        elif (hyperparameters[0]==1):
            # Decide the network architecture 
            print("Size:5 , 1 Hidden layer")
            input=int(train.shape[1])
            if(input<=5):
                h1=input-1
                latent=2
            else:    
        
                h1=int(input/2)
                latent=2 if int(h1/2)<2 else int(h1/2)
                latent=latent+1 if (latent%2!=0) else latent
            print(input,h1,latent,h1,input)
        
            # Setup the AE Architecture 
            
             
            input_layer = tensorflow.keras.Input(shape=train.shape[1:]) # Input Layer
            encoded = layers.Dense(h1, activation='relu' )(input_layer) # Code Layer 1
            encoded=layers.Dense(latent,activation='relu')(encoded) # Bottleneck
            decoded=layers.Dense(h1,activation='relu')(encoded)# Decode Layer 1
            decoded = layers.Dense(train.shape[1], activation='sigmoid',kernel_regularizer=kernel_regularizer)(decoded) # Output Layer

            autoencoder = tensorflow.keras.Model(input_layer, decoded)
            # above model maps an input to its reconstruction 
   
            #for hyperparameters in all_combinations:
                # Create and train an autoencoder with the given hyperparameters
            autoencoder.compile(optimizer=optimizer, loss='mse')
            print(hyperparameters)
            #print(hyperparameters[0],hyperparameters[1],hyperparameters[2])
            history=autoencoder.fit(train,train, 
                    epochs=100,
                    batch_size=32,
                    shuffle=True,
                    validation_data=(val_set, val_set)).history
             
            if np.mean(history['val_loss']) < best_loss:
                best_hyperparameters = hyperparameters
                best_loss = np.mean(history['val_loss'])
                best_model=autoencoder
                best_history=history

        
        elif (hyperparameters[0]==2):
            print("Size:7 , 2 Hidden layers")
            input=int(train.shape[1])
            if (input<10):
                h1=input-1
                h2=h1-1
                latent=2
            else:
                h1=int(input/2)
                h2=3 if int(h1/2)<3 else int(h1/2)
                latent=2 if int(h2/2)<2 else int(h2/2)
                latent=latent+1 if (latent%2!=0) else latent
            print(input,h1,h2,latent,h2,h1,input)
        
        
                # Setup the AE Architecture 
            
             
            input_layer = tensorflow.keras.Input(shape=train.shape[1:]) # Input Layer
            encoded = layers.Dense(h1, activation='relu' )(input_layer) # Code Layer 1
            encoded = layers.Dense(h2, activation='relu' )(encoded) # Code Layer 2
            encoded=layers.Dense(latent,activation='relu')(encoded) # Bottleneck
            decoded=layers.Dense(h2,activation='relu')(encoded)# Decode Layer 1
            decoded=layers.Dense(h1,activation='relu')(decoded)# Decode Layer 2
            decoded = layers.Dense(train.shape[1], activation='sigmoid',kernel_regularizer=kernel_regularizer)(decoded) # Output Layer

            autoencoder = tensorflow.keras.Model(input_layer, decoded)
            # above model maps an input to its reconstruction 
   
            #for hyperparameters in all_combinations:
                # Create and train an autoencoder with the given hyperparameters
            autoencoder.compile(optimizer=optimizer, loss='mse')
            print(hyperparameters)
            #print(hyperparameters[0],hyperparameters[1],hyperparameters[2])
            history=autoencoder.fit(train,train, 
                   epochs=100,
                    batch_size=32,
                    shuffle=True,
                    validation_data=(val_set, val_set)).history
             
            if np.mean(history['val_loss']) < best_loss:
                best_hyperparameters = hyperparameters
                best_loss = np.mean(history['val_loss'])
                best_model=autoencoder
                best_history=history
        
        
        elif (hyperparameters[0]==3):
            print("Size:9, Three Hidden Layers")  
            input=int(train.shape[1])
            if input<9:
                h1=input-1
                h2=h1-1
                h3=h2-1
                letent=2
            else:
            
                h1=int(input/2)
                h2=4 if int(h1/2)<4 else int(h1/2)
                h3=3 if int(h2/2)<3 else int(h2/2)
                latent=2 if int(h3/2)<2 else int(h3/2)
                latent=latent+1 if (latent%2!=0) else latent
            print(input,h1,h2,h3,latent,h3,h2,h1,input)
        
            
             # Setup the AE Architecture 
            
             
            input_layer = tensorflow.keras.Input(shape=train.shape[1:]) # Input Layer
            encoded = layers.Dense(h1, activation='relu' )(input_layer) # Code Layer 1
            encoded = layers.Dense(h2, activation='relu' )(encoded) # Code Layer 2
            encoded = layers.Dense(h3, activation='relu' )(encoded) # Code Layer 3
            encoded=layers.Dense(latent,activation='relu')(encoded) # Bottleneck
            decoded=layers.Dense(h3,activation='relu')(encoded)# Decode Layer 1
            decoded=layers.Dense(h2,activation='relu')(decoded)# Decode Layer 2
            decoded=layers.Dense(h1,activation='relu')(decoded)# Decode Layer 3
            decoded = layers.Dense(train.shape[1], activation='sigmoid',kernel_regularizer=kernel_regularizer)(decoded) # Output Layer

            autoencoder = tensorflow.keras.Model(input_layer, decoded)
            # above model maps an input to its reconstruction 
   
            #for hyperparameters in all_combinations:
                # Create and train an autoencoder with the given hyperparameters
            autoencoder.compile(optimizer=optimizer, loss='mse')
            print(hyperparameters)
            #print(hyperparameters[0],hyperparameters[1],hyperparameters[2])
            history=autoencoder.fit(train,train, 
                    epochs=100,
                    batch_size=32,
                    shuffle=True,
                    validation_data=(val_set, val_set)).history
             
            if np.mean(history['val_loss']) < best_loss:
                best_hyperparameters = hyperparameters
                best_loss = np.mean(history['val_loss'])
                best_model=autoencoder
                best_history=history
        
        elif (hyperparameters[0]==4):
            print("Size:11,  4 Hidden Layers") 
            input=int(train.shape[1])
            if input<20:
                continue # This code will be added later on based on need 
            
    
    return best_hyperparameters, best_loss ,best_model,best_history
        
     

In [17]:
best_hyperparameters, best_loss,best_model,best_history=grid_search(X_Positive_train,X_Positive_test,hyperparameter_space)

Size:3 , No Hidden layer
10 4 10
(0, <tensorflow.python.keras.optimizer_v2.adam.Adam object at 0x0000022BEDCA98E0>, None, 0.001)
Epoch 1/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0761 - val_loss: 0.0635
Epoch 2/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0593 - val_loss: 0.0533
Epoch 3/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0520 - val_loss: 0.0504
Epoch 4/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0494 - val_loss: 0.0487
Epoch 5/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0477 - val_loss: 0.0474
Epoch 6/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0465 - val_loss: 0.0466
Epoch 7/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0462 - val_loss: 0.0461
Epoch 8/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0455 - val_loss: 0.0457
Epoch 9/100
394/394 [==============================] - 1s 2ms/s

Epoch 80/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0437 - val_loss: 0.0440
Epoch 81/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0435 - val_loss: 0.0441
Epoch 82/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0433 - val_loss: 0.0441
Epoch 83/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0436 - val_loss: 0.0441
Epoch 84/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0433 - val_loss: 0.0441
Epoch 85/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0433 - val_loss: 0.0441
Epoch 86/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0430 - val_loss: 0.0441
Epoch 87/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0434 - val_loss: 0.0442
Epoch 88/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0437 - val_loss: 0.0441
Epoch 89/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0435

394/394 [==============================] - 1s 2ms/step - loss: 0.0781 - val_loss: 0.0785
Epoch 60/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0776 - val_loss: 0.0785
Epoch 61/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0782 - val_loss: 0.0785
Epoch 62/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0780 - val_loss: 0.0784
Epoch 63/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0782 - val_loss: 0.0785
Epoch 64/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0781 - val_loss: 0.0785
Epoch 65/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0779 - val_loss: 0.0785
Epoch 66/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0781 - val_loss: 0.0785
Epoch 67/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0781 - val_loss: 0.0784
Epoch 68/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0780 - val_loss: 

394/394 [==============================] - 1s 2ms/step - loss: 0.0692 - val_loss: 0.0701
Epoch 39/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0696 - val_loss: 0.0699
Epoch 40/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0694 - val_loss: 0.0704
Epoch 41/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0696 - val_loss: 0.0702
Epoch 42/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0691 - val_loss: 0.0701
Epoch 43/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0696 - val_loss: 0.0699
Epoch 44/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0700 - val_loss: 0.0699
Epoch 45/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0694 - val_loss: 0.0697
Epoch 46/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0693 - val_loss: 0.0701
Epoch 47/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0698 - val_loss: 

394/394 [==============================] - 1s 1ms/step - loss: 0.0786 - val_loss: 0.0788
Epoch 18/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0786 - val_loss: 0.0790
Epoch 19/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0786 - val_loss: 0.0789
Epoch 20/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0790 - val_loss: 0.0788
Epoch 21/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0783 - val_loss: 0.0790
Epoch 22/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0784 - val_loss: 0.0793
Epoch 23/100
394/394 [==============================] - 1s 1ms/step - loss: 0.0788 - val_loss: 0.0787
Epoch 24/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0785 - val_loss: 0.0787
Epoch 25/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0786 - val_loss: 0.0790
Epoch 26/100
394/394 [==============================] - 1s 1ms/step - loss: 0.0783 - val_loss: 

...

394/394 [==============================] - 1s 3ms/step - loss: 0.1148 - val_loss: 0.1152
Epoch 41/100
394/394 [==============================] - 1s 3ms/step - loss: 0.1146 - val_loss: 0.1147
Epoch 42/100
394/394 [==============================] - 1s 3ms/step - loss: 0.1138 - val_loss: 0.1141
Epoch 43/100
394/394 [==============================] - 1s 3ms/step - loss: 0.1132 - val_loss: 0.1136
Epoch 44/100
394/394 [==============================] - 1s 3ms/step - loss: 0.1126 - val_loss: 0.1131
Epoch 45/100
394/394 [==============================] - 1s 3ms/step - loss: 0.1121 - val_loss: 0.1125
Epoch 46/100
394/394 [==============================] - 1s 3ms/step - loss: 0.1115 - val_loss: 0.1120
Epoch 47/100
394/394 [==============================] - 1s 3ms/step - loss: 0.1107 - val_loss: 0.1115
Epoch 48/100
394/394 [==============================] - 1s 3ms/step - loss: 0.1109 - val_loss: 0.1110
Epoch 49/100
394/394 [==============================] - 1s 3ms/step - loss: 0.1100 - val_loss: 

394/394 [==============================] - 1s 3ms/step - loss: 0.0828 - val_loss: 0.0828
Epoch 20/100
394/394 [==============================] - 1s 3ms/step - loss: 0.0822 - val_loss: 0.0823
Epoch 21/100
394/394 [==============================] - 1s 3ms/step - loss: 0.0817 - val_loss: 0.0818
Epoch 22/100
394/394 [==============================] - 1s 3ms/step - loss: 0.0813 - val_loss: 0.0813
Epoch 23/100
394/394 [==============================] - 1s 3ms/step - loss: 0.0805 - val_loss: 0.0809
Epoch 24/100
394/394 [==============================] - 1s 3ms/step - loss: 0.0801 - val_loss: 0.0806
Epoch 25/100
394/394 [==============================] - 1s 3ms/step - loss: 0.0799 - val_loss: 0.0803
Epoch 26/100
394/394 [==============================] - 1s 3ms/step - loss: 0.0797 - val_loss: 0.0801
Epoch 27/100
394/394 [==============================] - 1s 3ms/step - loss: 0.0797 - val_loss: 0.0799
Epoch 28/100
394/394 [==============================] - 1s 3ms/step - loss: 0.0791 - val_loss: 

394/394 [==============================] - 1s 3ms/step - loss: 0.0781 - val_loss: 0.0784
Epoch 100/100
394/394 [==============================] - 1s 3ms/step - loss: 0.0780 - val_loss: 0.0784
Size:9, Three Hidden Layers
10 5 4 3 2 3 4 5 10
(3, <tensorflow.python.keras.optimizer_v2.gradient_descent.SGD object at 0x0000022BEDCA9AC0>, 'l2', 0.05)
Epoch 1/100
394/394 [==============================] - 1s 3ms/step - loss: 0.1452 - val_loss: 0.1159
Epoch 2/100
394/394 [==============================] - 1s 3ms/step - loss: 0.1091 - val_loss: 0.0961
Epoch 3/100
394/394 [==============================] - 1s 3ms/step - loss: 0.0927 - val_loss: 0.0868
Epoch 4/100
394/394 [==============================] - 1s 3ms/step - loss: 0.0849 - val_loss: 0.0825
Epoch 5/100
394/394 [==============================] - 1s 3ms/step - loss: 0.0815 - val_loss: 0.0805
Epoch 6/100
394/394 [==============================] - 1s 3ms/step - loss: 0.0796 - val_loss: 0.0795
Epoch 7/100
394/394 [===========================

394/394 [==============================] - 1s 3ms/step - loss: 0.0780 - val_loss: 0.0784
Epoch 79/100
394/394 [==============================] - 1s 3ms/step - loss: 0.0787 - val_loss: 0.0784
Epoch 80/100
394/394 [==============================] - 1s 3ms/step - loss: 0.0779 - val_loss: 0.0784
Epoch 81/100
394/394 [==============================] - 1s 3ms/step - loss: 0.0782 - val_loss: 0.0784
Epoch 82/100
394/394 [==============================] - 1s 3ms/step - loss: 0.0780 - val_loss: 0.0784
Epoch 83/100
394/394 [==============================] - 1s 3ms/step - loss: 0.0780 - val_loss: 0.0784
Epoch 84/100
394/394 [==============================] - 1s 3ms/step - loss: 0.0778 - val_loss: 0.0784
Epoch 85/100
394/394 [==============================] - 1s 3ms/step - loss: 0.0782 - val_loss: 0.0784
Epoch 86/100
394/394 [==============================] - 1s 3ms/step - loss: 0.0778 - val_loss: 0.0784
Epoch 87/100
394/394 [==============================] - 1s 3ms/step - loss: 0.0775 - val_loss: 

394/394 [==============================] - 1s 3ms/step - loss: 0.0726 - val_loss: 0.0734
Epoch 58/100
394/394 [==============================] - 1s 3ms/step - loss: 0.0729 - val_loss: 0.0733
Epoch 59/100
394/394 [==============================] - 1s 3ms/step - loss: 0.0723 - val_loss: 0.0733
Epoch 60/100
394/394 [==============================] - 1s 3ms/step - loss: 0.0725 - val_loss: 0.0732
Epoch 61/100
394/394 [==============================] - 1s 3ms/step - loss: 0.0725 - val_loss: 0.0731
Epoch 62/100
394/394 [==============================] - 1s 3ms/step - loss: 0.0725 - val_loss: 0.0730
Epoch 63/100
394/394 [==============================] - 1s 3ms/step - loss: 0.0722 - val_loss: 0.0729
Epoch 64/100
394/394 [==============================] - 1s 3ms/step - loss: 0.0724 - val_loss: 0.0729
Epoch 65/100
394/394 [==============================] - 1s 3ms/step - loss: 0.0718 - val_loss: 0.0729
Epoch 66/100
394/394 [==============================] - 1s 3ms/step - loss: 0.0721 - val_loss: 

In [18]:
best_hyperparameters, best_loss

((0,
  None,
  0.001),
 0.044795873314142226)

In [19]:
X_negative_train=X_negative[0:int(len(X_negative)*0.90)]
X_negative_test=X_negative[int((0.90*len(X_negative))):len(X_negative)-1]

In [20]:
best_hyperparameters2, best_loss2,best_model2,best_history2=grid_search(X_negative_train,X_negative_test,hyperparameter_space)

Size:3 , No Hidden layer
10 4 10
(0, <tensorflow.python.keras.optimizer_v2.adam.Adam object at 0x0000022BEDCA98E0>, None, 0.001)
Epoch 1/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0681 - val_loss: 0.0566
Epoch 2/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0558 - val_loss: 0.0545
Epoch 3/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0543 - val_loss: 0.0537
Epoch 4/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0536 - val_loss: 0.0534
Epoch 5/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0531 - val_loss: 0.0531
Epoch 6/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0532 - val_loss: 0.0529
Epoch 7/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0533 - val_loss: 0.0528
Epoch 8/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0530 - val_loss: 0.0527
Epoch 9/100
394/394 [==============================] - 1s 2ms/s

Epoch 80/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0516 - val_loss: 0.0524
Epoch 81/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0522 - val_loss: 0.0523
Epoch 82/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0522 - val_loss: 0.0523
Epoch 83/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0523 - val_loss: 0.0524
Epoch 84/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0524 - val_loss: 0.0523
Epoch 85/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0519 - val_loss: 0.0523
Epoch 86/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0521 - val_loss: 0.0523
Epoch 87/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0520 - val_loss: 0.0523
Epoch 88/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0521 - val_loss: 0.0523
Epoch 89/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0522

Epoch 59/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0437 - val_loss: 0.0440
Epoch 60/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0439 - val_loss: 0.0441
Epoch 61/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0433 - val_loss: 0.0441
Epoch 62/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0438 - val_loss: 0.0442
Epoch 63/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0435 - val_loss: 0.0441
Epoch 64/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0437 - val_loss: 0.0443
Epoch 65/100
394/394 [==============================] - ETA: 0s - loss: 0.043 - 1s 2ms/step - loss: 0.0438 - val_loss: 0.0441
Epoch 66/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0432 - val_loss: 0.0441
Epoch 67/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0434 - val_loss: 0.0442
Epoch 68/100
394/394 [==============================] - 1s

Epoch 38/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0695 - val_loss: 0.0705
Epoch 39/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0695 - val_loss: 0.0702
Epoch 40/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0700 - val_loss: 0.0702
Epoch 41/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0696 - val_loss: 0.0702
Epoch 42/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0698 - val_loss: 0.0706
Epoch 43/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0700 - val_loss: 0.0704
Epoch 44/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0699 - val_loss: 0.0702
Epoch 45/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0698 - val_loss: 0.0701
Epoch 46/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0695 - val_loss: 0.0702
Epoch 47/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0698

Epoch 17/100
394/394 [==============================] - ETA: 0s - loss: 0.079 - 1s 2ms/step - loss: 0.0793 - val_loss: 0.0789
Epoch 18/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0786 - val_loss: 0.0787
Epoch 19/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0789 - val_loss: 0.0786
Epoch 20/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0787 - val_loss: 0.0788
Epoch 21/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0789 - val_loss: 0.0785
Epoch 22/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0783 - val_loss: 0.0789
Epoch 23/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0789 - val_loss: 0.0789
Epoch 24/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0786 - val_loss: 0.0789
Epoch 25/100
394/394 [==============================] - 1s 2ms/step - loss: 0.0785 - val_loss: 0.0787
Epoch 26/100
394/394 [==============================] - 1s

...

Epoch 31/100
394/394 [==============================] - 1s 3ms/step - loss: 0.1204 - val_loss: 0.1196
Epoch 32/100
394/394 [==============================] - 1s 3ms/step - loss: 0.1200 - val_loss: 0.1190
Epoch 33/100
394/394 [==============================] - 1s 3ms/step - loss: 0.1187 - val_loss: 0.1183
Epoch 34/100
394/394 [==============================] - 1s 3ms/step - loss: 0.1181 - val_loss: 0.1177
Epoch 35/100
394/394 [==============================] - 1s 3ms/step - loss: 0.1177 - val_loss: 0.1171
Epoch 36/100
394/394 [==============================] - 1s 4ms/step - loss: 0.1171 - val_loss: 0.1166
Epoch 37/100
394/394 [==============================] - 1s 3ms/step - loss: 0.1163 - val_loss: 0.1160
Epoch 38/100
394/394 [==============================] - 1s 3ms/step - loss: 0.1157 - val_loss: 0.1154
Epoch 39/100
394/394 [==============================] - 1s 3ms/step - loss: 0.1155 - val_loss: 0.1148
Epoch 40/100
394/394 [==============================] - 1s 3ms/step - loss: 0.1148

394/394 [==============================] - 1s 3ms/step - loss: 0.0973 - val_loss: 0.0956
Epoch 10/100
394/394 [==============================] - 1s 3ms/step - loss: 0.0953 - val_loss: 0.0931
Epoch 11/100
394/394 [==============================] - 1s 3ms/step - loss: 0.0927 - val_loss: 0.0911
Epoch 12/100
394/394 [==============================] - 1s 3ms/step - loss: 0.0907 - val_loss: 0.0893
Epoch 13/100
394/394 [==============================] - 1s 3ms/step - loss: 0.0886 - val_loss: 0.0877
Epoch 14/100
394/394 [==============================] - 1s 3ms/step - loss: 0.0872 - val_loss: 0.0864
Epoch 15/100
394/394 [==============================] - 1s 3ms/step - loss: 0.0862 - val_loss: 0.0853
Epoch 16/100
394/394 [==============================] - ETA: 0s - loss: 0.0848- ETA: 0s -  - 1s 3ms/step - loss: 0.0848 - val_loss: 0.0843
Epoch 17/100
394/394 [==============================] - 1s 3ms/step - loss: 0.0840 - val_loss: 0.0835
Epoch 18/100
394/394 [==============================] - 1s

394/394 [==============================] - 1s 3ms/step - loss: 0.0779 - val_loss: 0.0782
Epoch 89/100
394/394 [==============================] - 1s 3ms/step - loss: 0.0781 - val_loss: 0.0782
Epoch 90/100
394/394 [==============================] - 1s 3ms/step - loss: 0.0777 - val_loss: 0.0782
Epoch 91/100
394/394 [==============================] - 1s 3ms/step - loss: 0.0786 - val_loss: 0.0782
Epoch 92/100
394/394 [==============================] - 1s 3ms/step - loss: 0.0780 - val_loss: 0.0782
Epoch 93/100
394/394 [==============================] - 1s 3ms/step - loss: 0.0784 - val_loss: 0.0782
Epoch 94/100
394/394 [==============================] - 1s 3ms/step - loss: 0.0782 - val_loss: 0.0782
Epoch 95/100
394/394 [==============================] - 1s 3ms/step - loss: 0.0781 - val_loss: 0.0782
Epoch 96/100
394/394 [==============================] - 1s 3ms/step - loss: 0.0781 - val_loss: 0.0782
Epoch 97/100
394/394 [==============================] - 1s 3ms/step - loss: 0.0777 - val_loss: 

394/394 [==============================] - 1s 3ms/step - loss: 0.0712 - val_loss: 0.0713
Epoch 67/100
394/394 [==============================] - 1s 3ms/step - loss: 0.0707 - val_loss: 0.0711
Epoch 68/100
394/394 [==============================] - 1s 3ms/step - loss: 0.0708 - val_loss: 0.0712
Epoch 69/100
394/394 [==============================] - 1s 3ms/step - loss: 0.0708 - val_loss: 0.0711
Epoch 70/100
394/394 [==============================] - 1s 3ms/step - loss: 0.0706 - val_loss: 0.0711
Epoch 71/100
394/394 [==============================] - 1s 3ms/step - loss: 0.0707 - val_loss: 0.0710
Epoch 72/100
394/394 [==============================] - 1s 3ms/step - loss: 0.0705 - val_loss: 0.0710
Epoch 73/100
394/394 [==============================] - 1s 3ms/step - loss: 0.0706 - val_loss: 0.0709
Epoch 74/100
394/394 [==============================] - 1s 3ms/step - loss: 0.0707 - val_loss: 0.0709
Epoch 75/100
394/394 [==============================] - 1s 4ms/step - loss: 0.0703 - val_loss: 

394/394 [==============================] - 1s 3ms/step - loss: 0.0782 - val_loss: 0.0782
Epoch 45/100
394/394 [==============================] - 1s 3ms/step - loss: 0.0783 - val_loss: 0.0782
Epoch 46/100
394/394 [==============================] - 1s 3ms/step - loss: 0.0785 - val_loss: 0.0782
Epoch 47/100
394/394 [==============================] - 1s 4ms/step - loss: 0.0781 - val_loss: 0.0782
Epoch 48/100
394/394 [==============================] - 1s 3ms/step - loss: 0.0782 - val_loss: 0.0782
Epoch 49/100
394/394 [==============================] - 1s 3ms/step - loss: 0.0780 - val_loss: 0.0782
Epoch 50/100
394/394 [==============================] - 1s 3ms/step - loss: 0.0784 - val_loss: 0.0782
Epoch 51/100
394/394 [==============================] - 1s 3ms/step - loss: 0.0782 - val_loss: 0.0782
Epoch 52/100
394/394 [==============================] - 1s 3ms/step - loss: 0.0781 - val_loss: 0.0782
Epoch 53/100
394/394 [==============================] - 1s 3ms/step - loss: 0.0779 - val_loss: 

In [21]:
best_hyperparameters2, best_loss2

((0,
  None,
  0.01),
 0.04405360318720341)

In [22]:
best_loss2

0.04405360318720341